# Mean, Median and Mode with Garmin Vivofit Data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import seaborn as sns
from scipy import stats

sns.set_palette(['#00A99D', '#F5CA0C', '#B6129F', '#76620C', '#095C57'])
# turn of data table rendering
pd.set_option('display.notebook_repr_html', False)
plt.style.use('ggplot')
pd.__version__
%matplotlib inline

## Data

In [ ]:
# Load 248 days of step data and vivofit goals
data = pd.read_csv('data\garmin-vivofit.csv', index_col='date')
data.drop(data.columns[[2, 3]],axis=1,inplace=True)
data.head()

In [ ]:
# Plot the steps and goal data
data.steps.plot(kind='area', figsize=(8,5), color='#00A99D', alpha=.5)
data.goal.plot()
plt.ylabel('steps per day')
plt.legend(loc='upper left')

## Calculate the mean
The arithmetic mean is the most commonly used measure of central tendency. The Greek letter $\mu$ (mu) is used to represent the population mean. To calculate the mean, we sum up all values $x_0+x_1+x_n$ and divide it by the number of values $n$.

$$\mu= \frac{1}{n}\sum_{i=0}^n x_i$$


In [ ]:
# Calculate the mean
x, n = 0.0, 0

for number_of_steps in data.steps:
    x += number_of_steps

n = len(data.steps)

mean = x / n
mean

In [ ]:
# We can also let pandas use NumPy's mean function to do the job
data.steps.mean()

In [ ]:
# Or we can call NumPy's mean function ourselves
np.mean(data.steps)

In [ ]:
# Use pandas to get the mean for all columns at ones
data.mean()

In [ ]:
# Plot the mean, togehter with the steps and goal data
data.steps.plot(kind='area', color='#00A99D', alpha=.5, figsize=(8,5))
data.goal.plot(legend=True)
plt.plot([0, len(data.steps)],[mean, mean])
plt.ylabel('steps per day')
plt.text(25, mean-1200, r'$\mu=' + str(int(math.floor(mean))) + '$', fontsize=14)
plt.legend(loc='upper left')

## Calculate the median
The median is often a better measure of central tendency when we have extreme outliers. The median is the value in the middle after we sort the data. This is why outliers do not influence the median as much as they do the mean. If the number of observations $n$ is even, we have to take the mean of the two middle values. We calculate for a zero based index.


$$n\ is\ odd:\ \ x_{median}=x_{\frac{n-1}{2}}$$

$$n\ is\ even:\ \ x_{median}=\frac{x_\frac{n-2}{2}+x_\frac{n}{2}}{2}$$


In [ ]:
median, n = 0.0, 0

# Get the number of observations
n = len(data.steps)
    
# order the data
ordered_data = data.steps.sort_values()

if n % 2 == 0:
    # n is even
    m1 = ordered_data.iloc[int((n - 2) / 2)]
    m2 = ordered_data.iloc[int((n / 2))]
    median = (m1 + m2) / 2.0
else:
    # n is odd
    median = ordered_data.ix[(n - 2) / 2.0]

median

In [ ]:
# Again, we can let pandas use NumPy's median function to do the job
data.steps.median()

In [ ]:
# Or we can call NumPy's median function ourselves
np.median(data.steps)

In [ ]:
# Use pandas to get the median for all columns at ones
data.median()

## Calculate the Mode
The mode is one or more values which occur most often in the series. This measure of central tendency is especially meaningful when you have a lot of repeated data points (like five-star ratings, day numbers). For the Garmin Vivofit data, however, this holds a problem, because we seldom have the same step count on a particular day. In that case we could categorize our data into range buckets, like 0-1000, 1000-2000 steps, and count the frequencies of the observations in the buckets. Choosing the right bucket size can be tricky, because we can obscure the data or miss the 'real' mode.

In [ ]:
# Let's create a lambda that assigns a bucket of size 1000 steps
# to each of the step count in the data set
bucket_size = 1000
bucket_calculator = lambda x: int(x) / bucket_size * bucket_size

data['bucket'] = data.steps.apply(bucket_calculator)
bucket_min = data.bucket.min()
bucket_max = data.bucket.max()
bins = int((bucket_max-bucket_min) / bucket_size)

data.bucket.hist(color='#00A99D', alpha=.5, bins=bins, figsize=(8,5))

print('Mode:', data.groupby('bucket').steps.count().idxmax(), \
      'with bucket size', bucket_size)

# Exploring the Variance and Standard Deviation

## Data
The data is a set of ten salaries, as used in the Udacity course 'Intro to Descriptive Statistics' lesson 4 on measures of variability.

In [ ]:
data = pd.DataFrame({'salaries': 
                     [33219, 36254, 38801, 46335, 46840, 
                      47596, 55130, 56863, 78070, 88830]})
data

In [ ]:
data.plot(kind='bar', color='#00A99D', alpha=.5)

## Calculate the Variance
The variance of a data set describes the average of the squared differences from the mean. In other words, it is a measure of how far each value in the data set is from the mean. The symbol for the variance of a population is $\sigma^2$ (sigma squared) and for a sample we use $s^2$. We calculate the variance by summing the squared difference from the mean for each value. For the population, we divide by the number of values $n$ in the data set.

$$population\ variance:\ \sigma^2=\frac{1}{n}\sum_{i=0}^n(x_i-\mu)^2$$

For the sample we divide the summed up values by the degrees of freedom $n-1$ (also called Bessel's correction). We use $\bar{x}$ (x bar) to symbolize our sample mean.

$$sample\ variance:\ s^2=\frac{1}{n-1}\sum_{i=0}^n(x_i-\bar{x})^2$$

In [ ]:
# To calculate the population variance
n = len(data.salaries)

# first calculate the mean
mean = data.salaries.mean()

# Sum up the squared differences from the mean
squared_deviations = 0
for v in data.salaries:
    squared_deviations += (v - mean) ** 2

population_variance = squared_deviations / n
population_variance

In [ ]:
# To calculate the variance if we only have a sample
# First calculate the degrees of freedom (apply Bessel's correction)
dof = n - 1
sample_variance = squared_deviations / dof
sample_variance

In [ ]:
# Of course we can use pandas to let NumPy do the job for us
# The ddof parameter stands for Delta Degrees of Freedom
population_variance = data.salaries.var(ddof=0)
sample_variance = data.salaries.var() # ddof=1 by default in pandas

population_variance, sample_variance

In [ ]:
# Or call the NumPy var function ourselves
population_variance = np.var(data.salaries) # ddof=0 by default in NumPy
sample_variance = np.var(data.salaries, ddof=1)

population_variance, sample_variance

## Calculate the Standard Deviation
The standard deviation is a widely used normalized measure of spread of a data set. Unlike the variance, the standard deviation is using the same scale as our values; dollars in this case. In a normal distribution, about 95% of the values are within two standard deviations of the mean. We use the Greek letter sigma $\sigma$ to symbolize the population standard deviation. 

$$population\ standard\ deviation:\ \sigma=\sqrt{\frac{1}{n}\sum_{i=0}^n(x_i-\mu)^2}\ \ =\ \ \sqrt{\sigma^2}$$

We use the lowercase letter $s$ if we indicate the sample standard deviation.

$$sample\ standard\ deviation:\ s=\sqrt{\frac{1}{n-1}\sum_{i=0}^n(x_i-\bar{x})^2}\ \ =\ \ \sqrt{s^2}$$

In [ ]:
# To calculate the population standard deviation
# we first need to calculate the population variance again
n = len(data.salaries)

# first calculate the mean
mean = data.salaries.mean()

# Sum up the squared differences from the mean
squared_deviations = 0
for v in data.salaries:
    squared_deviations += (v - mean) ** 2

population_variance = squared_deviations / n

# Square the variance
population_standard_deviation = math.sqrt(population_variance)
population_standard_deviation

In [ ]:
# To calculate the sample standard deviation
# First calculate the degrees of freedom (apply Bessel's correction)
dof = n - 1
sample_variance = squared_deviations / dof

# Square the variance
sample_standard_deviation = math.sqrt(sample_variance)
sample_standard_deviation

In [ ]:
# Now let's use pandas to let NumPy do the job for us
population_standard_deviation = data.salaries.std(ddof=0)
sample_standard_deviation = data.salaries.std()

population_standard_deviation, sample_standard_deviation

In [ ]:
# Or call the NumPy std function ourselves
population_standard_deviation = np.std(data.salaries)
sample_standard_deviation = np.std(data.salaries, ddof=1)

population_standard_deviation, sample_standard_deviation

# Exploring the Standard Normal Distribution

## Data

In [ ]:
#We use a fictional data set of 10000 averge number of Facebook friends.
facebook_mu = 190.0
facebook_sigma = 36.0
facebook_friends = np.random.normal(facebook_mu, facebook_sigma, 10000)

# show first 12 samples
facebook_friends[:12]

## Using the Probability Density Function
The total area under the Probability Density Function (pdf) is always 1.0. Roughly 68% of the values is within 1 standard deviation from the mean. About 95% falls within two standard deviations. We can determine the probability of finding a given value in the distribution by using the pdf. 

Let's say someone's got 227 Facebook friends. What is the probability of having this or less number of Facebook friends?

In [ ]:
# First take a look at the pdf and especially the green area under
# the curve containing the probability of 227 Facebook friends or less.
x = 227.0
sns.distplot(facebook_friends, label='Facebook friends', kde=False, 
             fit=stats.norm, color='w')
plt.text(x+5, .0003, '$x$='+str(x))

x_plot = np.linspace(min(facebook_friends), x, 1000)
y_plot = stats.norm.pdf(x_plot, facebook_mu, facebook_sigma)
plt.fill_between(x_plot,  y_plot)
c=plt.legend()

In [ ]:
# To calculate the probability, we need the z score.
zscore = (x - facebook_mu) / facebook_sigma
zscore

In [ ]:
# Calculate the probability by calling stats.norm.cdf
# This is a computational z table lookup
p = stats.norm.cdf(zscore)
p

So this means the probability of people having 227 Facebook friends or less is about 85%. Since the area under the curve adds up to 1, we can say that the probability of people having a value more than 227 Facebook friends is $1-p$.

In [ ]:
# Probability of having a value more than 227
1 - p

## From probability back to the actual value
Let's assume we have a 21% chance of having a certain number of Facebook friends or more. What is the minimum number of Facebook friends we have in this case?

We use the ppf function (inverse cdf or $F^{-1}$) - from probability to z score

In [ ]:
p = 1 - .21
z = stats.norm.ppf(p)
z

In [ ]:
# From z score to number of Facebook friends
x = z * facebook_sigma + facebook_mu
x

In [ ]:
# The green area under the curve containing the probability 
# of (roughly) 206 Facebook friends or more.
sns.distplot(facebook_friends, label='Facebook friends', kde=False, 
             fit=stats.norm, color='w')
plt.text(x+5, .0003, '$x$='+str(int(x)))

x_plot = np.linspace(x, max(facebook_friends), 1000)
y_plot = stats.norm.pdf(x_plot, facebook_mu, facebook_sigma)
plt.fill_between(x_plot,  y_plot)
c=plt.legend()

## Calculate probability in between two values
What is the probability of people having between 120 and 170 Facebook friends?

In [ ]:
# We want to know the proportion of the green area under the curve.
x1 = 120.0
x2 = 170.0
sns.distplot(facebook_friends, label='Facebook friends', kde=False, 
             fit=stats.norm, color='w')
plt.text(x1+5, .0003, '$x_1$='+str(x1))
plt.text(x2+5, .0003, '$x_2$='+str(x2))

x_plot = np.linspace(x1, x2, 1000)
y_plot = stats.norm.pdf(x_plot, facebook_mu, facebook_sigma)
plt.fill_between(x_plot,  y_plot)
c=plt.legend()

In [ ]:
# First we need the z score of x1
z1 = (x1 - facebook_mu) / facebook_sigma
z1

In [ ]:
# Then we calculate the probability for value x1 or less
p1 = stats.norm.cdf(z1)
p1

In [ ]:
# Now we calculate the z score for x2
z2 = (x2 - facebook_mu) / facebook_sigma
z2

In [ ]:
# and agian the probabilty for value x2 or less
p2 = stats.norm.cdf(z2)
p2

In [ ]:
# So the probability of having between x1 and x2 Facebook friends is
# the probability having x2 minus the probability having x1
p2 - p1

# Homework

Remember an example with S&P 500 stock index. Let return is distributed as a normal random variable with $\mu = 0.001$ and $\sigma = 0.01$. In the following lines you can find the real data of this index.
Return values are also calculated. 

You should find out 
 * What is the real percent of losses greater than 5% per day?
 * What is the sample mean and variance? Compare it to the given in Example.
 * What is the median return?
 * What is a VaR for more risky strategy with 90% confidence (we expect the dayli loss < 10%) and investment of \$100000

In [ ]:
# Load and plot Close values of index
data = pd.read_csv('data/GSPC.csv',sep=',')
data.Close.plot()

In [ ]:
# calculate the index returns like in the example
fund_return = pd.Series(data.Close[1:].values - data.Close[:-1].values)/1000
fund_return.plot()

In [ ]:
fund_return.hist(bins=32)

In [ ]:
# YOUR CODE HERE